In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('data.json').read()
intents = json.loads(data_file)

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
nltk.download('wordnet')
nltk.download('omw-1.4')
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


77 documents
14 classes ['Director', 'Location', 'Self', 'courses', 'fee', 'fest', 'goodbye', 'greeting', 'information', 'noanswer', 'options', 'placement', 'seats', 'thanks']
98 unique lemmatized words ["'s", ',', '.', 'about', 'address', 'and', 'anyone', 'are', 'available', 'awesome', 'b', 'b.tech', 'be', 'build', 'by', 'bye', 'can', 'chatting', 'college', 'collge', 'come', 'company', 'computer', 'contact', 'could', 'course', 'created', 'cse', 'day', 'develop', 'director', 'do', 'engineering', 'event', 'fee', 'fest', 'for', 'function', 'give', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'in', 'information', 'intake', 'is', 'itm', 'know', 'later', 'list', 'location', 'made', 'mba', 'me', 'name', 'next', 'nice', 'of', 'offer', 'offered', 'or', 'parampara', 'place', 'placement', 'provide', 'record', 'science', 'seat', 'see', 'self', 'structure', 'support', 'tech', 'tell', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'vis

In [7]:
pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))

In [8]:
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-8-8451cd400d46>:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [9]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [10]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)
print("model created")

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
16/16 [==============================] - 1s 5ms/step - loss: 2.6721 - accuracy: 0.0779
Epoch 2/200
16/16 [==============================] - 0s 4ms/step - loss: 2.6024 - accuracy: 0.1429
Epoch 3/200
16/16 [==============================] - 0s 3ms/step - loss: 2.4912 - accuracy: 0.1818
Epoch 4/200
16/16 [==============================] - 0s 3ms/step - loss: 2.4520 - accuracy: 0.2338
Epoch 5/200
16/16 [==============================] - 0s 3ms/step - loss: 2.4373 - accuracy: 0.1818
Epoch 6/200
16/16 [==============================] - 0s 4ms/step - loss: 2.2504 - accuracy: 0.3506
Epoch 7/200
16/16 [==============================] - 0s 3ms/step - loss: 2.1046 - accuracy: 0.4026
Epoch 8/200
16/16 [==============================] - 0s 3ms/step - loss: 1.9853 - accuracy: 0.3896
Epoch 9/200
16/16 [==============================] - 0s 3ms/step - loss: 1.9265 - accuracy: 0.4156
Epoch 10/200
16/16 [==============================] - 0s 4ms/step - loss: 1.8266 - accuracy: 0.4805
Epoch 11/